In [21]:
import pickle
import os
from sklearn.svm import SVC
import cv2
import numpy as np
from sklearn.metrics import classification_report
# from face_process import Face_affine

In [22]:
import numpy as np
import argparse
import dlib
import cv2
import imutils
import matplotlib.pyplot as plt


class Face_affine():
    def __init__(self, image_path=None,path_model='./models/shape_predictor_68_face_landmarks.dat'):
        # self.detector = dlib.get_frontal_face_detector()
        self.predictor = dlib.shape_predictor(path_model)
        self.face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
        # self.image = cv2.imread(image_path)
        self.image = image_path
        # self.image = imutils.resize(self.image, width=500)
        self.gray = None
    
    def update_image(self,image_path):
        self.image = image_path
        self.gray = cv2.cvtColor(self.image, cv2.COLOR_BGR2GRAY)
    
    def rect_to_bb(self,rect):
        x = rect.left()
        y = rect.top()
        w = rect.right() - x
        h = rect.bottom() - y
        return (x, y, w, h)

    def shape_to_np(self, shape, dtype="int"):
        coords = np.zeros((68, 2), dtype=dtype)
        for i in range(0, 68):
            coords[i] = (shape.part(i).x, shape.part(i).y)
        return coords
    
    def get_reacts2(self):
        rects = self.detector(self.gray, 1)
        rects = [(rect.left(), rect.top(), 
                  rect.right()-rect.left(), 
                  rect.bottom()-rect.top()) for rect in rects]
        
        return rects
    
    def get_reacts(self):
        # detect faces in the grayscale image
        # rects = self.detector(self.gray, 1)
        rects = self.face_classifier.detectMultiScale(self.gray, scaleFactor=1.1, minNeighbors=9, minSize=(64, 64))
        return rects
    
    def get_shapes(self,rects):
        shapes = []
        for (i, rect) in enumerate(rects):
            # print(rect)
            rect = dlib.rectangle(rect[0], rect[1], rect[0]+rect[2], rect[1]+rect[3])
            shape = self.predictor(self.gray, rect)
            shape = self.shape_to_np(shape)
            shapes.append(shape)
        return shapes
    
    def get_face(self,shape):
        (x, y, w, h) = cv2.boundingRect(shape)
        roi = self.image[y:y+h, x:x+w]
        return roi
    
    def get_face2(self,image,rect):
        (x, y, w, h) = rect
        roi = image[y:y+h, x:x+w]
        return roi
    
    def get_shape_drop(self,image,shape,rect):
        (x, y, w, h) =  rect
        shape_drop = shape - np.array([x,y])
        image_drop = image[y:y+h, x:x+w]
        return image_drop, shape_drop
    
    
    
    def rotating_scaling(self,image,shape, rect):
        desiredFaceWidth, desiredFaceHeight= image.shape[1], image.shape[0]
        # compute the center of mass for each eye
        leftEyeCenter = shape[36:42].mean(axis=0).astype("int")
        rightEyeCenter = shape[42:48].mean(axis=0).astype("int")
        # compute the angle between the eye centroids
        dY = rightEyeCenter[1] - leftEyeCenter[1]
        dX = rightEyeCenter[0] - leftEyeCenter[0]
        angle = np.degrees(np.arctan2(dY, dX)) 
        # compute the desired right eye x-coordinate based on the desired x-coordinate of the left eye
        desiredLeftEye=(0.35, 0.35)
        desiredRightEyeX = 1.0 - desiredLeftEye[0]
        dist = np.sqrt((dX ** 2) + (dY ** 2)) # distance between the eyes
        desiredDist = (desiredRightEyeX - desiredLeftEye[0]) * desiredFaceWidth # desired distance between the eyes
        scale = desiredDist / dist
        # compute center (x, y)-coordinates 
        # between the two eyes in the input image
        eyesCenter = ((leftEyeCenter[0] + rightEyeCenter[0]) / 2, (leftEyeCenter[1] + rightEyeCenter[1]) / 2)
        # grab the rotation matrix for rotating and scaling the face
        M = cv2.getRotationMatrix2D(eyesCenter, angle, scale)
        # update the translation component of the matrix
        tX = desiredFaceWidth * 0.5
        tY = desiredFaceHeight * desiredLeftEye[1]
        M[0, 2] += (tX - eyesCenter[0])
        M[1, 2] += (tY - eyesCenter[1])
        # apply the affine transformation
        (w, h) = (desiredFaceWidth, desiredFaceHeight)
        output = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC)
        # return the aligned face
        output = cv2.cvtColor(output, cv2.COLOR_BGR2RGB)
        # draw bounding box react of face detection after alignment
        (x, y, w, h) = rect
        x = int(x/scale)
        y = int(y/scale)
        w = int(w*scale)
        h = int(h*scale)
        # cv2.rectangle(output, (x, y), (x + w, y + h), (0, 255, 0), 2)
        
        return output, (x, y, w, h)
    
    def drop_padding_black(self,image):
        # drop padding black
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        _, thresh = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY)
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cnt = contours[0]
        x, y, w, h = cv2.boundingRect(cnt)
        crop = image[y:y+h, x:x+w]
        return crop
    
    def drop_all_black(self,image):
        # drop padding black
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        _, thresh = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY)
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cnt = contours[0]
        print(cnt[0][0][0], cnt[0][0][1])
        cv2.drawContours(image, contours[0], -1, (0,255,0), 3)
        cv2.circle(image, (cnt[0][0][0], cnt[0][0][1]), 5, (0,0,255), -1)
        # drop padding all countour
        a = []
        x, y, w, h = cv2.boundingRect(cnt)
        crop = image[y:y+h, x:x+w]
        return crop

In [23]:
# load the model from disk
filename = './models/model_face_not_41.sav'
clf = pickle.load(open(filename, 'rb'))
# load pca
filename = './models/pca_face_not_41.sav'
pca = pickle.load(open(filename, 'rb'))

def normailze_data_face(data):
    data = data.astype('float64')
    data /= 255
    return data

In [24]:
# video = cv2.VideoCapture('./data_true/IMG_5016.MOV')
video = cv2.VideoCapture(0)

 
# face_classifier = cv2.CascadeClassifier(
#     cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
# )



openface = Face_affine()
cnt = 0
while True:
    ret, frame = video.read()
    if ret:
        # cv2.imwrite('./data_true/image_test/frame.jpg',frame)
        openface.update_image(frame)
        
        reacts = openface.get_reacts()
        if len(reacts) == 0:
            cv2.imshow('frame', frame)
            print('no face')
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
            continue
        shapes = openface.get_shapes(reacts)
        faces = []
        for i in range(len(reacts)):
            (x, y, w, h) = reacts[i]
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            image_drop,shape_drop = openface.get_shape_drop(openface.gray,shapes[i],reacts[i])
            image_rotating_scaling,rect_scale = openface.rotating_scaling(image_drop,shape_drop,reacts[i])
            img = openface.drop_padding_black(image_rotating_scaling)
            img = cv2.resize(image_drop, (64, 64))
            faces.append(img)
        faces = np.array(faces)
        faces = normailze_data_face(faces)
        faces = faces.reshape((faces.shape[0],faces.shape[1] * faces.shape[2]))
        
        faces = pca.transform(faces)
        y_pred = clf.predict(faces)
        for i in range(len(y_pred)):
            if y_pred[i] == 1:
                cv2.putText(frame, 'MinhAnh', (reacts[i][0], reacts[i][1]), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2, cv2.LINE_AA)
            else:
                cv2.putText(frame, 'Unknow', (reacts[i][0], reacts[i][1]), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2, cv2.LINE_AA)
        print(y_pred)
        cnt += 1
        
        cv2.imshow('frame', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
